In [1]:
import joblib
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score


### 모델 불러오기

In [2]:
model = joblib.load('pkl/xgboost_081.pkl')

### 테스트 데이터 불러오기

In [3]:
# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    # "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"../data/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            print(file_path)
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

../data/test/2.신용정보/201807_test_신용정보.parquet
credit_test_07 is loaded from ../data/test/2.신용정보/201807_test_신용정보.parquet
../data/test/2.신용정보/201808_test_신용정보.parquet
credit_test_08 is loaded from ../data/test/2.신용정보/201808_test_신용정보.parquet
../data/test/2.신용정보/201809_test_신용정보.parquet
credit_test_09 is loaded from ../data/test/2.신용정보/201809_test_신용정보.parquet
../data/test/2.신용정보/201810_test_신용정보.parquet
credit_test_10 is loaded from ../data/test/2.신용정보/201810_test_신용정보.parquet
../data/test/2.신용정보/201811_test_신용정보.parquet
credit_test_11 is loaded from ../data/test/2.신용정보/201811_test_신용정보.parquet
../data/test/2.신용정보/201812_test_신용정보.parquet
credit_test_12 is loaded from ../data/test/2.신용정보/201812_test_신용정보.parquet
../data/test/3.승인매출정보/201807_test_승인매출정보.parquet
sales_test_07 is loaded from ../data/test/3.승인매출정보/201807_test_승인매출정보.parquet
../data/test/3.승인매출정보/201808_test_승인매출정보.parquet
sales_test_08 is loaded from ../data/test/3.승인매출정보/201808_test_승인매출정보.parquet
../data/test/3.승인매출정보/2018

0

In [4]:
info_categories = ["credit", "sales", "billing", "balance", "channel", "marketing", "performance"] # "customer", 

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

In [5]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


# customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

credit_test_df is created with shape: (600000, 42)
sales_test_df is created with shape: (600000, 406)
billing_test_df is created with shape: (600000, 46)
balance_test_df is created with shape: (600000, 82)
channel_test_df is created with shape: (600000, 105)
marketing_test_df is created with shape: (600000, 64)
performance_test_df is created with shape: (600000, 49)


0

In [6]:
#### Test ####

customer_test_df = pd.read_csv('../clean_data/userinfo_test_preprocessed.csv')
customer_test_df['기준년월'] = pd.to_datetime(customer_test_df['기준년월'])
customer_test_df['기준년월'] = customer_test_df['기준년월'].dt.strftime('%Y%m').astype(int)

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df",    "Step2"),
    ("billing_test_df",  "Step3"),
    ("balance_test_df",  "Step4"),
    ("channel_test_df",  "Step5"),
    ("marketing_test_df","Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: test_step1, shape: (600000, 106)
Step2 저장 완료: test_Step2, shape: (600000, 510)
Step3 저장 완료: test_Step3, shape: (600000, 554)
Step4 저장 완료: test_Step4, shape: (600000, 634)
Step5 저장 완료: test_Step5, shape: (600000, 737)
Step6 저장 완료: test_Step6, shape: (600000, 799)
최종 저장 완료: test_최종, shape: (600000, 846)


In [14]:
# 저장된 encoders 불러오기 (딕셔너리: {컬럼명: LabelEncoder})
encoders = joblib.load('pkl/encoders2.pkl')
le_target = joblib.load('pkl/le_target2.pkl')
zero_importance_features = joblib.load("pkl/zero_importance_features.pkl")

In [17]:
# test 데이터 복사
X_test = test_df.copy()
X_test = X_test.drop(columns = zero_importance_features)
X_test = X_test.drop(columns=["ID"])

# 범주형 컬럼만 encoders를 기준으로 변환
for col, le in encoders.items():
    if col in X_test.columns:
        # 예측 시 test 데이터에 학습 때 없던 범주가 있을 수 있음 → 추가
        unseen = set(X_test[col]) - set(le.classes_)
        if unseen:
            le.classes_ = np.append(le.classes_, list(unseen))

        X_test[col] = le.transform(X_test[col])


In [18]:
# feature_cols = [col for col in test_df.columns if col not in ["ID", "Segment"]]
# X_test = X_test[feature_cols].copy()

In [ ]:
# import joblib
# joblib.dump(feature_cols, 'pkl/feature_cols.pkl')

['pkl/feature_cols.pkl']

In [19]:
na_df = test_df.isnull().sum().reset_index()
na_df.columns = ['column', 'na_count']
na_df = na_df[na_df['na_count'] > 0].sort_values(by='na_count', ascending=False)
na_df['na_ratio'] = na_df['na_count'] / len(test_df)

# 전체 출력
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(na_df)

,column,na_count,na_ratio
564,연체일자_B0M,598605,0.997675
287,_3순위여유업종,594565,0.990942
293,_3순위납부업종,578711,0.964518
285,_2순위여유업종,575821,0.959702
281,_3순위교통업종,510729,0.851215
291,_2순위납부업종,510271,0.850452
283,_1순위여유업종,498313,0.830522
374,최종카드론_대출일자,497538,0.829230
372,최종카드론_신청경로코드,489783,0.816305
371,최종카드론_금융상환방식코드,489771,0.816285


In [20]:
print(X_test.isnull().sum())
# print(X)
for col in na_df["column"]:
    X_test[col] = X_test[col].fillna(0)

기준년월                        0
남녀구분코드                      0
연령                          0
회원여부_이용가능                   0
회원여부_이용가능_CA                0
회원여부_이용가능_카드론               0
소지여부_신용                     0
소지카드수_유효_신용                 0
소지카드수_이용가능_신용               0
입회일자_신용                     0
입회경과개월수_신용                  0
회원여부_연체                     0
이용거절여부_카드론                  0
동의여부_한도증액안내                 0
수신거부여부_TM                   0
수신거부여부_DM                   0
수신거부여부_메일                   0
수신거부여부_SMS                  0
가입통신회사코드                    0
탈회횟수_누적                     0
최종탈회후경과월                    0
탈회횟수_발급6개월이내                0
탈회횟수_발급1년이내                 0
거주시도명                       0
직장시도명                       0
마케팅동의여부                     0
유효카드수_신용체크                  0
유효카드수_신용                    0
유효카드수_신용_가족                 0
유효카드수_체크                    0
이용가능카드수_신용체크                0
이용가능카드수_신용                  0
이용가능카드수_신용_가족               0
이용가능카드수_체크

In [21]:
print(X_test.head())

     기준년월  남녀구분코드    연령  회원여부_이용가능  회원여부_이용가능_CA  회원여부_이용가능_카드론  소지여부_신용  \
0  201807       1  40.0          1             1              0        1   
1  201807       1  60.0          1             1              0        1   
2  201807       1  40.0          1             1              1        1   
3  201807       2  40.0          1             1              1        1   
4  201807       2  40.0          1             0              1        1   

   소지카드수_유효_신용  소지카드수_이용가능_신용   입회일자_신용  입회경과개월수_신용  회원여부_연체  이용거절여부_카드론  \
0            2              2  20140501          51        0           1   
1            1              1  20160201          30        0           0   
2            2              2  20180301           5        0           0   
3            1              1  20120701          73        0           0   
4            1              1  20031201         176        0           0   

   동의여부_한도증액안내  수신거부여부_TM  수신거부여부_DM  수신거부여부_메일  수신거부여부_SMS  가입통신회사코드  \
0            

In [22]:
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

In [23]:
submission = test_data.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]
submission.to_csv('../submit/0328_2.csv',index=False)